<a href="https://colab.research.google.com/github/dowrave/Data_Analysis_Projects/blob/main/Dacon_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install scikit-optimize

In [ ]:
import pandas as pd
from pandas import CategoricalDtype
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from skopt import BayesSearchCV

pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv')

In [ ]:
train.head(2)

In [ ]:
train.info()

# 데이터 탐색

In [ ]:
train.hist(bins = 20, figsize = (20, 20))

## 질적 데이터

- 전체 데이터 중 유망주의 비율 : $0.361$

In [ ]:
train['Prospect'].sum() / train['Prospect'].count()

#### 1. `PreferredFoot, WorkRate`
선요약 )  
- 특히 `WorkRate`의 경우는 높을수록 좋을 것이라는 생각을 하고 접근했지만, 가장 좋은 수치는 `Medium, Medium`에서 나왔음
- `PreferredFoot`의 경우 또한 `Left`에서 희소성이 있을 거라 생각하고 접근했지만, 그렇지는 않았음
- 둘 다 특별히 고칠 요소는 없는 것으로 보임


In [ ]:
# 위 그래프는 이렇게 그려도 되지 않을까?
sns.set_style('whitegrid')
plt.figure(figsize = (10, 10))
grid = sns.FacetGrid(data = train, col = 'Prospect')
grid.map(sns.countplot, 'PreferredFoot', palette= 'tab10')

In [ ]:
# 각 발 전체 인원 중 유망주 비율
print(train[(train['PreferredFoot'] == 'Left')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['PreferredFoot'] == 'Left']['ID'].count(),
      train[(train['PreferredFoot'] == 'Right')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['PreferredFoot'] == 'Right']['ID'].count(),
)
# 전체 인원 중 각 발의 비율
print(train[train['PreferredFoot'] == 'Left']['ID'].count() / train['ID'].count(),
      train[train['PreferredFoot'] == 'Right']['ID'].count() / train['ID'].count(),
)
# 유망주 전체 인원 중 각 발의 비율
print(train[(train['PreferredFoot'] == 'Left')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['Prospect'] == 1]['ID'].count(),
      train[(train['PreferredFoot'] == 'Right')
      & (train['Prospect'] == 1)]['ID'].count() / train[train['Prospect'] == 1]['ID'].count(),
)

1. 각 발에 대해 전체 인원 중 유망주 비율
- 왼발 : $0.33$
- 오른발 : $0.37$

2. 전체 인원과 각 발의 비율
- 왼발 : $0.24$
- 오른발 : $0.75$

3. 유망주 전체 인원과 각 발의 비율
- 왼발 : $0.22$
- 오른발 : $0.78$

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (10, 10))

for i, val in enumerate(['AttackingWorkRate', 'DefensiveWorkRate']):
  if i == 0:
    color = np.array(sns.color_palette('OrRd'))[[1, 3, 5], :]
  elif i == 1:
    color = np.array(sns.color_palette('Blues'))[[1, 3, 5], :]
  
  (train[val].value_counts()[['Low', 'Medium', 'High']]
                          .to_frame()
                          .T
                          .plot(kind = 'bar', 
                                stacked = True, 
                                rot = 0,
                                ax = ax[i][0],
                                color = color)
                          )
  ax[i][0].legend().set_visible(False)

  (train[train['Prospect'] == 1][val].value_counts()[['Low', 'Medium', 'High']]
                                                  .to_frame()
                                                  .T
                                                  .plot(kind = 'bar', 
                                                        stacked = True, 
                                                        rot = 0,
                                                        ax = ax[i][1],
                                                        color = color
  ))

  ax[i][1].set_ylim(ax[i][0].get_ylim())

In [ ]:
# 위 사항은 수치적으로 보고 싶음 : WorkRate에는 총 9가지 유형이 있는데, 이에 따른 생존율을 비교해보자
temp = (train[['ID', 'AttackingWorkRate', 'DefensiveWorkRate', 'Prospect']]
                                                              .groupby(['AttackingWorkRate', 'DefensiveWorkRate', 'Prospect'])['ID']
                                                              .count()
                                                              .to_frame()
                                                              .reset_index().rename(columns = {"ID" : 'counts'})
)
temp.head(2)

In [ ]:
sns.catplot(data = temp, 
            x = 'AttackingWorkRate', 
            y = 'counts', 
            hue = 'DefensiveWorkRate', 
            col = 'Prospect', 
            kind = 'bar', 
            order = ['Low', 'Medium', 'High'], 
            hue_order = ['Low', 'Medium', 'High'],
            ax = ax)
plt.suptitle("ATT / DEF WorkRate and The Num of Prospect", y = 1.05, fontsize = 15)
plt.tight_layout()

In [ ]:
temp['rates'] = temp['counts'] / temp.groupby(['AttackingWorkRate', 'DefensiveWorkRate'])['counts'].transform("sum")
temp_hm = temp[temp['Prospect'] == 1][:]
temp_hm = temp_hm.pivot(index = 'AttackingWorkRate', columns = 'DefensiveWorkRate', values = 'rates').loc[['High', 'Medium', 'Low'], ['Low', 'Medium', 'High']]

In [ ]:
plt.figure(figsize = (7, 7))
sns.heatmap(data = temp_hm, annot = True, fmt = '.3f', cmap = 'Blues', vmin = 0, vmax = 1)
plt.title("Att / Def WorkRate and Prospect Rate")

- `Low, Low`에 있는 요소는 샘플이 1개니까 크게 고려할 요소는 아님
- 애초에 예상한 건 `오른쪽 위로 갈수록 활동량이 좋으니 Prospect 비율이 높을 것이다` 였는데, 그렇지는 않은 것으로 나타남

#### 2. `Position`

In [ ]:
train['Position'].unique()

In [ ]:
# 먼저 시각화 하고 시작함
position_order = ['GK', 'LB', 'CB', 'RB', 'LWB', 'CDM', 'RWB', 'LM', 'CM', 'RM', 'CAM', 'LW', 'CF', 'RW', 'ST']
pos = (train[['Position', 'Prospect']].value_counts()
                                .to_frame()
                                .reset_index()
                                .rename(columns = {0 : 'counts'})
                                .pivot(index = 'Position', 
                                       columns = 'Prospect', 
                                       values = 'counts')
                                .loc[position_order]
    )
pos

In [ ]:
fig, ax = plt.subplots(figsize = (10, 5))
pos.plot(kind = 'bar', stacked = True, ax = ax)
ax.set_title("Position and Prospect")
ax.set_axisbelow(True)
plt.grid(True, axis = 'y')

- CAM, CB, ST, GK, RM 순으로 데이터의 양이 많음
- CF, LW, RW는 정말 적은 양을 차지함
  - 그러나 0과 1에 대한 label이 모두 있기 때문에 포지션에서 배제하지는 않겠음

## 양적 데이터

In [ ]:
plt.figure(figsize = (7, 7))

corr = train.corr()
mask = np.zeros_like(corr, dtype = bool)
mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것

sns.heatmap(data = corr, cmap = 'coolwarm', mask = mask) 

- 위 히트맵에서, 일부 필요없는 부분을 제외하고 시각화 함
- 보고 싶은 건 2가지임
  1. 각종 스탯 간의 관계
  2. 각종 스탯과 `Position Rating`의 관계
- 또한, 

In [ ]:
border = train.corr().columns.get_loc('GKReflexes') + 1 # 47

In [ ]:
# 1. 스탯 간의 관계 보기

# 근데 그냥 0 이상으로 두는게 낫지 않을까 싶으요?
# 요건 솔직히 어떻게 해놔야 될지 모르겠다. 다음에 보도록 하자.
corr1 = train.corr().iloc[:border, :border]
mask = np.zeros_like(corr1, dtype = bool)
mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것


plt.figure(figsize = (12, 12))
sns.heatmap(data = corr1, 
            cmap = 'PRGn', 
            mask = (mask 
                   # | np.where(corr1 > 0.5, 0, 1)
                    ), 
            vmin = -1, 
            vmax = 1) 

- 전체적으로 봤을 때, 모든 값을 모델에 포함한다면 중복되는 영역이 생김
  - 예를 들면 ~~Total에 들어가는 값에는 중복이 있을 것
  - 또한 `Rating`에 대한 수치도 중복이 있을 것이다

In [ ]:
corr2 = train.corr().iloc[:border, border:]
# mask = np.zeros_like(corr2, dtype = bool)
# mask[np.triu_indices_from(mask)] = True # 원래 행렬의 윗부분 절반을 1로 만드는 것

plt.figure(figsize = (12, 12))
sns.heatmap(data = corr2, 
            cmap = 'PRGn', 
            # mask = np.where(corr2 > 0.3, 0, 1),
            vmin = -1, 
            vmax = 1) 

- 이렇게 놓고 봤을 때 명백히 나뉘는 구역이 3가지가 있음
  - `LM, CM, RM` 영역 및 이보다 공격적인 위치
  - `LWB, CDM, RWB` 영역 ~ 수비수 영역까지 수비적인 위치
  - `GK`
- 이렇게 나눠지는 영역에서도 `CM`이나 `CDM`은 다른 경향을 보임


In [ ]:
foot_order = ['Left', 'Right']
work_rate_order = ['Low', 'Medium', 'High']

def column_label_order(df, col, order_lst):

  LE = LabelEncoder()
  LE.fit(df[col])
  LE.classes_ = np.array(order_lst)
  df[col] = LE.transform(df[col])


column_label_order(train, 'PreferredFoot', foot_order)
column_label_order(train, 'AttackingWorkRate', work_rate_order)
column_label_order(train, 'DefensiveWorkRate', work_rate_order)

In [ ]:
train.columns

In [ ]:
train_ab = train.iloc[:, np.r_[0: train.columns.get_loc('DefensiveWorkRate') + 1, train.columns.get_loc('Crossing') : train.columns.get_loc('GKReflexes') + 1, -1]]
train_ab.head(2)

In [ ]:
train_ab_avg = (train_ab.iloc[:, 1:].groupby(['Position', 'Prospect'])
                                    .mean()
                                    .reset_index()
                                    .set_index(['Position', 'Prospect'])
                                    .T
                )
train_ab_avg.head(2)

- 위 표를 보면 가장 의외인 지점은, 같은 포지션에서도 `Prospect = 0`의 스탯이 더 높은 수치를 나타내는 곳이 많다.
- `Age`는 `Prospect`를 결정하는 중요한 요인이라고 생각됨.

In [ ]:
sns.countplot(data = train, x = 'Age', hue = 'Prospect')

## 중복을 생각해 배제했던 Rating이나 Total에 대한 정보 보기



### 1. Rating
- 선수의 주 포지션과 주 포지션 레이팅, 유망 여부의 df를 따로 분리해서 보겠음
  - 즉 자기 포지션의 레이팅과 유망 여부가 관련이 있는가를 보는 것

In [ ]:
train['Position'].unique()

In [ ]:
pos_rat = train.iloc[:, np.r_[0, 4, train.columns.get_loc('STRating') : train.columns.get_loc('Prospect') + 1]]
GK = pos_rat[pos_rat['Position'] == 'GK'][['ID', 'Position', 'GKRating', 'Prospect']].rename(columns = {'GKRating' : 'POSRating' })
ST = pos_rat[pos_rat['Position'] == 'ST'][['ID', 'Position', 'STRating', 'Prospect']].rename(columns = {'STRating' : 'POSRating' })
RB = pos_rat[pos_rat['Position'] == 'RB'][['ID', 'Position', 'RBRating', 'Prospect']].rename(columns = {'RBRating' : 'POSRating' })
LB = pos_rat[pos_rat['Position'] == 'LB'][['ID', 'Position', 'LBRating', 'Prospect']].rename(columns = {'LBRating' : 'POSRating' })
CDM = pos_rat[pos_rat['Position'] == 'CDM'][['ID', 'Position', 'CDMRating', 'Prospect']].rename(columns = {'CDMRating' : 'POSRating' })
CAM = pos_rat[pos_rat['Position'] == 'CAM'][['ID', 'Position', 'CAMRating', 'Prospect']].rename(columns = {'CAMRating' : 'POSRating' })
CM = pos_rat[pos_rat['Position'] == 'CM'][['ID', 'Position', 'CMRating', 'Prospect']].rename(columns = {'CMRating' : 'POSRating' })
RM = pos_rat[pos_rat['Position'] == 'RM'][['ID', 'Position', 'RMRating', 'Prospect']].rename(columns = {'RMRating' : 'POSRating' })
LM = pos_rat[pos_rat['Position'] == 'LM'][['ID', 'Position', 'LMRating', 'Prospect']].rename(columns = {'LMRating' : 'POSRating' })
RWB = pos_rat[pos_rat['Position'] == 'RWB'][['ID', 'Position', 'RWBRating', 'Prospect']].rename(columns = {'RWBRating' : 'POSRating' })
LWB = pos_rat[pos_rat['Position'] == 'LWB'][['ID', 'Position', 'LWBRating', 'Prospect']].rename(columns = {'LWBRating' : 'POSRating' })
RW = pos_rat[pos_rat['Position'] == 'RW'][['ID', 'Position', 'RWRating', 'Prospect']].rename(columns = {'RWRating' : 'POSRating' })
CF = pos_rat[pos_rat['Position'] == 'CF'][['ID', 'Position', 'CFRating', 'Prospect']].rename(columns = {'CFRating' : 'POSRating' })
LW = pos_rat[pos_rat['Position'] == 'LW'][['ID', 'Position', 'LWRating', 'Prospect']].rename(columns = {'LWRating' : 'POSRating' })
pos_rat = pd.concat([GK, ST, RB, LB, CDM, CAM, CM, RM, LM, RWB, LWB, RW, CF, LW])

In [ ]:
pos_rat_stt = pos_rat.groupby(['Position', 'Prospect'])['POSRating'].mean().reset_index().rename(columns = {"POSRating" : 'mean_score'})

pos_rat_stt.columns

In [ ]:
fig, ax = plt.subplots(figsize = (10, 7))

sns.barplot(data = pos_rat_stt,
            x = 'Position',
            y = 'mean_score',
            hue = 'Prospect',
            ax = ax)
plt.gca().legend().set_visible(False)
ax.set_ylim([5, 7])
ax.set_title('Position Rating and Prospect')

- 역시 `Prospect = 1`에서 전반적인 스탯이 더 낮게 나옴
  - `LW`가 특이한 모습을 보이지만 이는 샘플 수가 적은 것에서 기인한다고 판단함
  - 포지션 별로 평균 나이를 뽑아도 되겠지만 위에서 나이에 대한 결론을 미리 냈기 때문에 따로 진행하지 않겠음
- `CF`의 경우 `Prospect = 1`이 없기 때문에 따로 처리할 필요가 있음

In [ ]:
sns.heatmap(data = train.corr().iloc[[train.corr().columns.get_loc('STRating'), train.corr().columns.get_loc('CFRating'), train.corr().columns.get_loc('CAMRating')], [train.corr().columns.get_loc('STRating'), train.corr().columns.get_loc('CFRating'), train.corr().columns.get_loc('CAMRating')]],
            annot = True,
            fmt = '.3f'
)

- `CF`는 `CAM`과의 상관계수가 더 높기 때문에 `CAM`으로 처리하겠음

### 2. Total

In [ ]:
pos_tot = train.iloc[:, np.r_[0, 4, train.columns.get_loc('PaceTotal') : train.columns.get_loc('PhysicalityTotal') + 1, -1]]
pos_tot_stt = pos_tot.groupby(['Position', 'Prospect'])['PaceTotal', 'ShootingTotal', 'PassingTotal', 'DribblingTotal', 'DefendingTotal', 'PhysicalityTotal'].mean().reset_index()
pos_tot_stt = pos_tot_stt.melt(id_vars = ['Position', 'Prospect'])
pos_tot_stt

In [ ]:
pos_tot_stt

In [ ]:
sns.catplot(data = pos_tot_stt,
        x = 'Prospect',
        y = 'value',
        row = 'Position',
        hue = 'variable',
        kind = 'bar'
)

- 역시 `Prospect = 1`에서 전반적으로 낮은 스탯이 나옴



## 선호하는 발과, 포지션 간의 관계
- 위에선 선호하는 발의 비율만을 봤지만, 통념상 왼쪽 측면엔 왼발 선수, 오른쪽 측면엔 오른쪽발 선수를 선호하는 경향이 있다.
- 정말 그런지 살펴보고 그렇다면 선호하는 발에 대한 특성을 추가해준다

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
temp = train[['ID', 'Position', 'PreferredFoot', 'Prospect']]
temp['Position'].unique()

In [ ]:
def get_hor_pos(x):
  if x in ['LB', 'LM', 'LWB', 'LW']:
    return 0
  elif x in ['RB', 'RM', 'RWB', 'RW']:
    return 1
  else:
    return 2

temp.loc[:, 'horizontal_pos'] = temp['Position'].apply(lambda x : get_hor_pos(x))
temp = temp.groupby(['horizontal_pos', 'PreferredFoot', 'Prospect'])['ID'].count().to_frame().reset_index().rename(columns = {'ID' : 'counts'})

In [ ]:
sns.catplot(data = temp,
            x = 'horizontal_pos',
            y = 'counts',
            row = 'PreferredFoot',
            col = 'Prospect',
            kind = 'bar',
            order = ['LEFT', 'CENTER', 'RIGHT'],
            ax = ax
            )


- 포지션이 Center인 경우는 큰 상관이 없지만, 측면에 서는 경우는 관련이 있음을 알 수 있음
- 유망한지 여부와 관련이 없을 수도 있지만 중요한 특성이라고 생각되어 기존 주 포지션을 횡 / 종으로 나누는 것을 생각해봄직함

## 수직적인 위치
- LF, RF, CF는 없는 포지션이다~ 생각하겠음
  - 실제로 앞의 2개는 없고, 뒤의 1개는 매우 적음
  

In [ ]:
temp = train.corr()
a = temp.columns.get_loc('STRating')
b = temp.columns.get_loc('GKRating')

plt.figure(figsize = (12, 12))
sns.heatmap(data = temp.iloc[np.r_[a:a+2, a+5 : b+1], np.r_[a:a+2, a+5 : b+1]],
            annot = True,
            fmt = '.3f',
            vmin = 0.95,
            vmax = 1,
            cmap = 'Blues'
)

- 가장 충돌이 있을만한 지점들이 윙백이랑 수미, 풀백의 높이정보임
  - 윙백 > 풀백인 건 알겠는데, 수미를 놓으면 애매함
  - 근데 윙백이랑 풀백이 비슷하게 나온단 말임?
  - 수미는 풀백에 더 가까움
- 방법 1. `ST` - `LW, RW, CAM, LM, RM` - `CM` - `LWB/RWB` - `LB/CDM/RB` - `CB` - `GK` 순으로 놓는다


In [ ]:

def get_vert_pos(x):
  if x in ['ST']:
    return 6
  elif x in ['LW', 'RW', 'CAM', 'LM', 'RM']:
    return 5
  elif x in ['CM']:
    return 4
  elif x in ['LWB', 'RWB']:
    return 3
  elif x in ['LB',' CDM', 'RB']:
    return 2
  elif x in ['CB']:
    return 1
  else:
    return 0

train['vert_pos'] = train['Position'].apply(lambda x : get_vert_pos(x))
train['hor_pos'] = train['Position'].apply(lambda x : get_hor_pos(x))
train = train.drop('Position', axis = 1)


In [ ]:
train

In [ ]:
# 포지션 비슷한 정도를 파악할 때 PCA 쓸 수 있지 않을까?

- 방법 2. `ST` - `LW, RW, CAM, LM, RM` - `CM` - `LWB/RWB/LB/CDM/RB` - `CB` - `GK` 순으로 놓는다

- 방법 3. 작은 소수점들을 뭉뚱그리지 않고 그냥 데이터를 그대로 넣는다

------------------------------------
## 구분선 & 위 내용 요약
1. `Prospect = 1`에서 레이팅이든 토탈 스탯이든 전반적으로 낮은 경향을 보인다.
  - 이는 유망한 선수들의 나이가 전반적으로 더 어린 데에서 기인한다. 평균으로 생각해도 마찬가지
2. `CF`는 양이 적고, 무조건 `Prospect = 0`이기 때문에 별도의 처리를 해준다
  - `df.corr()`을 돌렸을 때, `ST`와 `CAM`중 `CAM`에 대한 상관계수가 더 높기 때문에 **데이터 처리 시 `CF`는 `CAM`으로 처리한다**
3. 포지션을 횡 / 종으로 나눈다
  - 측면에 서는 경우 선호하는 발과 위치 간의 연관 관계가 있기 때문이다.
  - 종의 경우 6가지 vs 5가지 중 고민 중
------------------------------------

# 전처리


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv', index_col = 'ID')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv', index_col = 'ID')

In [ ]:

# CF -> CAM
train[train['Position'] == 'CF'][:] = 'CAM'

# Position -> Horizontal + Vertical
train['vert_pos'] = train['Position'].apply(lambda x : get_vert_pos(x))
train['hor_pos'] = train['Position'].apply(lambda x : get_hor_pos(x))
train = train.drop('Position', axis = 1)


# CF -> CAM
test[test['Position'] == 'CF'][:] = 'CAM'

# Position -> Horizontal + Vertical
test['vert_pos'] = test['Position'].apply(lambda x : get_vert_pos(x))
test['hor_pos'] = test['Position'].apply(lambda x : get_hor_pos(x))
test = test.drop('Position', axis = 1)

print(train.shape, test.shape)

### LabelEncoder

In [ ]:
# 이거는 모델에 넣기 전에 진행하면 됨 - 먼저 나눠버리면 X축이 뭘 의미하는지 모르게 됨

foot_order = ['Left', 'Right']
work_rate_order = ['Low', 'Medium', 'High']

def column_label_order(df, col, order_lst):

  LE = LabelEncoder()
  LE.fit(df[col])
  LE.classes_ = np.array(order_lst)
  df[col] = LE.transform(df[col])
  test[col] = LE.transform(test[col]) # 테스트 데이터에도 적용함

column_label_order(train, 'PreferredFoot', foot_order)
column_label_order(train, 'AttackingWorkRate', work_rate_order)
column_label_order(train, 'DefensiveWorkRate', work_rate_order)


### OneHotEncoder

In [ ]:
# 범주형 데이터들 원핫인코딩
train_X = train.iloc[:, np.r_[:3, train.columns.get_loc('Crossing'):train.columns.get_loc('GKReflexes') + 1]]
train_y = train['Prospect']


for i in ['PreferredFoot', 'AttackingWorkRate', 'DefensiveWorkRate', 'Position']:
  onehot = pd.get_dummies(train[i], 
                #  drop_first = True,
                prefix = i
                )
  train_X = train_X.join(onehot)

### 공통
- 인코딩된 값들은 따로 표준화를 진행하지 않음
- 나이 같은 값은 minMaxScaling이 나은 것 같은데

In [ ]:
# 여기부턴 공통
train_X = train.iloc[:, np.r_[:6, train.columns.get_loc('Crossing'):train.columns.get_loc('GKReflexes') + 1, -2, -1]]
train_y = train.loc[:, 'Prospect']
test = test.iloc[:, np.r_[:6, test.columns.get_loc('Crossing'):test.columns.get_loc('GKReflexes') + 1, -2, -1]]

train_X.shape, test.shape

In [ ]:
# 나머지 모든 데이터를 평균 0, 표준편차 1 사이로 표준화함
# 나중에 정규화(minmax)로도 해보자
ss = StandardScaler()
ss.fit_transform(train_X.iloc[:, np.r_[:3, test.columns.get_loc('Crossing'):train_X.columns.get_loc('GKReflexes') + 1]])
# train_X.iloc[:, :train_X.columns.get_loc('GKReflexes') + 1] = ss.transform(train_X.iloc[:, :train_X.columns.get_loc('GKReflexes') + 1])

# 훈련 세트에 적용한 스케일러를 테스트 세트에도 동일하게 적용한다
test.iloc[:, np.r_[:3, test.columns.get_loc('Crossing'):train_X.columns.get_loc('GKReflexes') + 1]] = ss.transform(test.iloc[:, np.r_[:3, test.columns.get_loc('Crossing'):train_X.columns.get_loc('GKReflexes') + 1]])

In [ ]:
train_X.shape, train_y.shape, test.shape

# 모델 적용

## 1. 랜덤 포레스트

- HPO : RandomizedSearchCV

In [ ]:
# 하이퍼 파라미터 파라미터 튜닝하고 ㄱ
clf = RandomForestClassifier()


params_rs = {'max_depth' : list(range(5, 10)),
          'n_estimators' : list(range(50, 300)),
          'bootstrap' : [True, False],
          'max_features' : ['sqrt', None]
          }

# RandomizedSearch
model = RandomizedSearchCV(estimator = clf,
                           param_distributions = params_rs,
                           n_iter = 30,
                           cv = 5,
                           n_jobs = -1, 
                           verbose = 1,
                            random_state = 0)

model.fit(train_X, train_y)

In [ ]:
model.best_params_

- HPO : BayesSearchCV

In [ ]:
clf = RandomForestClassifier()

params_bs = {'max_depth' : (5, 10),
          'n_estimators' : (25, 500),
          }

# BayesSearch
model = BayesSearchCV(estimator = clf,
                           search_spaces = params_bs,
                           n_iter = 25,
                           cv = 5,
                           n_jobs = -1, 
                           verbose = 1,
                           random_state = 0)

In [ ]:
model.fit(train_X, train_y)

In [ ]:
# model.cv_results_['params'], model.cv_results_['mean_test_score'] # 결과값 시각화
print(model.best_params_)
print(model.best_score_)

In [ ]:
model.best_params_

In [ ]:
pred = model.predict(test)
pred

In [ ]:
submit = pd.DataFrame({'Prospect' : pred}, index = test.index)
submit
submit.to_csv('RF_Bayes_GetPosition3_221119.csv')

In [ ]:
# Ordinial Encoder를 써본다든가
# Age Height Weight를 써본다든가
# 뭐 등등이 있겠쥬?

## 2. DNN
- 가장 간단한 타입을 써보겠읍니다

In [ ]:
import tensorflow as tf 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

sub_X, val_X, sub_y, val_y = train_test_split(train_X, train_y, test_size = 0.2, random_state = 0, stratify = train_y)
sub_X.shape, sub_y.shape, val_X.shape, val_y.shape

In [ ]:
model = tf.keras.Sequential(
                            [tf.keras.layers.Dense(128 , 
                                                  #  activation = 'relu',
                                                  #  input_dim = 37
                                                   ),
                             tf.keras.layers.LeakyReLU(alpha = 0.1),

                             tf.keras.layers.Dense(64, 
                                                  #  activation = 'relu'
                                                   ), 
                             tf.keras.layers.LeakyReLU(alpha = 0.1),

                             tf.keras.layers.Dense(32, 
                                                  #  activation = 'relu'
                                                   ),
                             tf.keras.layers.LeakyReLU(alpha = 0.1),
                             
                             tf.keras.layers.Dense(1, activation = 'sigmoid')]
                            )

model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy',
              # loss = 'mse',
              metrics = ['Accuracy'])

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.25,
                              patience = 2,
                              min_lr = 0.00005)

es = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1, restore_best_weights = True)

model.fit(sub_X, sub_y,
          epochs = 30,
          batch_size = 64,
          validation_data = (val_X, val_y),
          callbacks = [reduce_lr, es])

In [ ]:
pred = np.round_(model.predict(test)).flatten()

In [ ]:
submit = pd.DataFrame({'Prospect' : pred}, index = test.index)
submit
submit.to_csv('submit_LabelEncoder_NN.csv')